In [1]:
# This script downaloads all data stored in a production build and creates json files that can be used for local development

In [1]:
import os
import json
import re
from rethinkdb import RethinkDB
from dotenv import dotenv_values

In [2]:
#PATHS
PATH_OUT = './../datasets/production/'
PATH_ENV = './../env/.env.tunnel'

In [ ]:
# connect to rethinkdb database and download actual entities table
# this needs a tunnel to be initialized with the command ssh -L 28016:<IP>:28015 <user>@<IP>
r = RethinkDB()
config = dotenv_values(PATH_ENV)

conn = r.connect(host=config["DB_HOST"], port=config["DB_PORT"], password=config["DB_PASS"], db=config["DB_NAME"])

# get all tables
entities = r.table('entities').without("createdAt", "updatedAt", "entities").coerce_to('array').run(conn)
relations = r.table('relations').coerce_to('array').run(conn)
documents = r.table('documents').coerce_to('array').run(conn)

# fix problem Array over size limit `100000`.  To raise the number of allowed elements, modify the `array_limit` option to `.run` (not available in the Data Explorer), or use an index in:
# r.table('audits').coerce_to('array')
audits = []
for i in range(0, 100000, 1000):
    audits += r.table('audits').skip(i).limit(1000).coerce_to('array').run(conn)

users = r.table('users').coerce_to('array').run(conn)

conn.close()

# for each table create JSON files if they do not exist save them in the data folder
if not os.path.exists(PATH_OUT):
    os.makedirs(PATH_OUT)

with open(PATH_OUT + 'entities.json', 'w') as f:
    json.dump(list(entities), f)

with open(PATH_OUT + 'relations.json', 'w') as f:
    json.dump(list(relations), f)

# fixes Object of type datetime is not JSON serializable
for audit in audits:
    audit['date'] = str(audit['date'])
with open(PATH_OUT + 'audits.json', 'w') as f:

    json.dump(list(audits), f)

with open(PATH_OUT + 'users.json', 'w') as f:
    json.dump(list(users), f)

# with open(PATH_OUT + 'documents.json', 'w') as f:
#     json.dump(list(documents), f)


TypeError: Object of type datetime is not JSON serializable